In [ ]:
from urban_sdk_homework.modules.traffic.services import TrafficService
from urban_sdk_homework.modules.traffic.models import Link

# Test the traffic service.  (This isn't stricly necessary.  It just tells us the database is connected.)
service = TrafficService.connect()
print("Traffic service connected successfully!")

In [11]:
# Set up.
# MAPBOX_TOKEN = "your-mapbox-token"
BASE_URL = "http://localhost:8000"

In [ ]:
# Request Aggregated Data
import json
import requests

# TODO: Add support for day and period names.
# TODO: Add offset and limit parameters.
params = {
    "day": "Monday",    # Monday
    "period": "Evening"  # "AM Peak"
}
response = requests.get(f"{BASE_URL}/aggregates/", params=params)
geojson_data = response.json()
# print(json.dumps(geojson_data, indent=2))

# Transform the data into a GeoJSON feature collection.
features = [
    {
        "type": "Feature",
        "geometry": f["geom"],
        "properties": {
            "average_speed": f["speed"],
            "road_name": f["road_name"]
        }
    } for f in geojson_data
]
feature_collection = {
    "type": "FeatureCollection",
    "features": features
}
print(json.dumps(feature_collection, indent=2))

In [20]:
# MapboxGL was having some problems.  First an incompatibility with ipython,
# but then once we resolved that, MapboxGL was drawing the LineStrings as
# polygons.  I'm assuming that getting MapboxGL working isn't the main goal,
# so let's move to the next cell and try Folium.

# # Visualize in Mapbox
# from mapboxgl.viz import ChoroplethViz
# from mapboxgl.utils import create_color_stops

# features = [
#     {
#         "type": "Feature",
#         "geometry": f["geom"],
#         "properties": {
#             "average_speed": f["speed"],
#             "road_name": f["road_name"]
#         }
#     } for f in geojson_data
# ]
# feature_collection = {
#     "type": "FeatureCollection",
#     "features": features
# }
# viz = ChoroplethViz(
#     feature_collection,
#     access_token=MAPBOX_TOKEN,
#     color_property="average_speed",
#     color_stops=create_color_stops([10, 20, 30, 40, 50], colors="Reds"),
#     center=(-81.6557, 30.3322),
#     zoom=11,
#     line_width=1.5,
#     opacity=0.8
# )
# viz.show()

# import json
# print(json.dumps(feature_collection, indent=2))

In [ ]:
import folium
import json

# Create map centered on Jacksonville
m = folium.Map(location=[30.3322, -81.6557], zoom_start=11)

# Define color function based on speed
def get_color(speed):
    if speed < 20:
        return 'red'
    elif speed < 40:
        return 'orange'
    elif speed < 60:
        return 'yellow'
    else:
        return 'green'

# Add GeoJSON with proper styling for LineStrings
folium.GeoJson(
    feature_collection,
    style_function=lambda feature: {
        'color': get_color(feature['properties']['average_speed']),
        'weight': 4,
        'opacity': 0.8
    },
    popup=folium.GeoJsonPopup(
        fields=['road_name', 'average_speed'],
        aliases=['Road:', 'Speed (mph):'],
        labels=True
    )
).add_to(m)

# Display in Jupyter (if running in notebook)
m